In [1]:
import re
import random
import csv
import pandas as pd


In [ ]:
ARABIC_LETTERS = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"

# حذف التشكيل
def remove_tashkeel(text):
    return re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)

# تنظيف النص
def clean_text(text):
    text = remove_tashkeel(text)
    text = ' '.join([word for word in text.split() if not re.search(r'[a-zA-Z]', word)])
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_punctuation(text):
    return re.sub(r'[،؛؟!.\',"]', '', text)  


In [ ]:
def introduce_typos(text, num_typos=3):
    words = text.split()
    new_words = words.copy()

    
    typo_count = min(num_typos, len(words))
    typo_indices = random.sample(range(len(words)), typo_count)

    for idx in typo_indices:
        word = new_words[idx]
        if not word:
            continue

        typo_type = random.choice(['replace', 'delete', 'insert-middle', 'swap', 'insert-end'])
        chars = list(word)

        if typo_type == 'replace' and chars:
            i = random.randint(0, len(chars) - 1)
            chars[i] = random.choice(ARABIC_LETTERS)

        elif typo_type == 'delete' and len(chars) > 1:
            i = random.randint(0, len(chars) - 1)
            del chars[i]

        elif typo_type == 'insert-middle':
            i = random.randint(0, len(chars))
            chars.insert(i, random.choice(ARABIC_LETTERS))

        elif typo_type == 'swap' and len(chars) > 1:
            i = random.randint(0, len(chars) - 2)
            chars[i], chars[i + 1] = chars[i + 1], chars[i]

        elif typo_type == 'insert-end':
            chars.append(random.choice(ARABIC_LETTERS))

        new_words[idx] = ''.join(chars)

    return ' '.join(new_words)

In [ ]:
input_file = "ara_wikipedia_2021_30K-sentences.txt"
output_pairs = []

with open(input_file, "r", encoding="utf-8") as infile:
    for line in infile:
        correct = clean_text(line)
        if correct:
            noisy = introduce_typos(correct)
            if noisy != correct:
                correct = remove_punctuation(correct) 
                noisy = remove_punctuation(noisy)  
                output_pairs.append((correct, noisy))  

In [ ]:

csv_file = "formatted_dataset.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["correct", "incorrect"]) 
    for correct, noisy in output_pairs:
        writer.writerow([correct, noisy])

print(f"Done{csv_file}")

In [ ]:
df = pd.read_csv(csv_file)


print(df.head(10))

                                             correct  \
0              أغسطس هي عالمة فقه في اللغة الأسبانية   
1                     هو عسكري و سياسي و نبيل إسباني   
2            أطروحة تعليمية عن مكان الفرد في المجتمع   
3   مستعمرة إيطالية تأسست عام بعد ضم أراضي الإمبر...   
4   تستند الأوصاف في على مجموعة لغة تتجاوز مليون ...   
5          ١ فترة ظهور ثقافة الموشي في موتشي المبكرة   
6                                       يناير أبريل    
7    والتي يمكن وصفها بأنها الفحص الدوري مع متعدد...   
8  مراكز ثقافيةدار شباب مكتبة بلدية مكتبة جامعية ...   
9                                    ذهب بعيار عشرين   

                                           incorrect  
0          ذ أغسطس هي عظالمة كفقه في اللغة الأسبانية  
1                     هو عسكري و سياسس و نبيل إبساني  
2          أطروحة تعليميةص عنع مكان نلفرد في المجتمع  
3   سمتعمرة إيطالية تأسست عام بعد ضم أراضي الإمبر...  
4   تستند الأوصاف في على مجموعة لغة تتجاوز ملينو ...  
5        ١ فترا ظهور ثقافة الموشيا في موتشيث المبكرة 